In [ ]:
from image_preprocessing import preprocess
from object_detection import detect_objects
from object_rendering import draw_boxes

In [ ]:
image_path = './sample-images/cat.jpg'

prediction_url = ''

In [ ]:
image_data = preprocess(image_path)
objects = detect_objects(
    image_data, prediction_url, token=''
)
draw_boxes(image_path, *objects)